In [1]:
import pandas as pd

In [2]:
city_list = [
    "atlanta",
    "boston",
    "chicago",
    "los_angeles",
    "new_york",
    "philadelphia",
    "portland",
    "san_francisco",
    "washington",
]
df_list = []
for city in city_list:
    weather = pd.read_parquet(f"../00_source/{city}_weather.parquet").reset_index(
        drop=True
    )
    weather = weather[
        weather["datatype"].isin(["PRCP", "TMAX", "TMIN", "SNOW", "SNWD"])
    ]
    weather["Year"] = pd.DatetimeIndex(weather["date"]).year
    weather["Month Numeric"] = pd.DatetimeIndex(weather["date"]).month
    weather["Day"] = pd.DatetimeIndex(weather["date"]).day
    weather = weather.fillna(0)
    temp = weather.pivot_table(
        values="value",
        index=["City", "Year", "Month Numeric", "Day"],
        columns=["datatype"],
    ).reset_index()
    temp["Precipitation"] = temp["PRCP"] / (25.4)
    temp["Snowfall"] = temp["SNOW"] / (25.4)
    temp["Snow Depth"] = temp["SNWD"] / (25.4)
    temp["Maximum Temperature"] = ((temp["TMAX"] / 10) * 9 / 5) + 32
    temp["Minimum Temperature"] = ((temp["TMAX"] / 10) * 9 / 5) + 32
    temp = (
        temp.groupby(["City", "Year", "Month Numeric"])[
            [
                "Maximum Temperature",
                "Minimum Temperature",
                "Precipitation",
                "Snowfall",
                "Snow Depth",
            ]
        ]
        .mean()
        .reset_index()
    )
    df_list.append(temp)
    pass


In [3]:
weather_final = pd.concat(df_list)

In [4]:
weather_final.to_parquet(
    "../20_intermediate_data/weather.parquet", engine="fastparquet"
)